### Script to generate FATES parameters using LHS

In [6]:
from scipy.stats import qmc
import numpy as np
import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
import shutil
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                                                                                                                                                                       
import tempfile 
import random
import re
import modp as mp

In [5]:
random.seed(32)

#### Read in min and max values for each parameter and pft 
- Adapted from work by Rachel Ward (parameter ranges and inequalities). 
- Where there are inequalities between PFTx and PFTy, sample PFTx parameter and then use a scalar to generate PFTy parameter. -- If parameter ranges are the same ignore the inequality
- Add some noise to the inequalities so that inter-PFT trait correlations are not too strict

### Some parameters are help equal for all conifers

There are some parameters that we keep equal among the conifers. We want the value pulled for pine from the Latin hypercube to apply to all conifers.

In [7]:
equal_among_conifers = ["fates_recruit_seed_alloc","fates_recruit_seed_alloc_mature"]

In [3]:
param_ranges_full = pd.read_csv('/global/homes/j/jneedham/DBEN/Parameter_files/BCI_ensemble_params_trimmed.csv')
param_ranges = param_ranges_full[['param', 'value_min', 'value_max', 'pft', 'organ']]

# number of parameters
n_params = len(param_ranges)

# number of PFTs - some are global so subtract one
n_pfts = len(pd.unique(param_ranges['pft'])) - 1

param_names = list(param_ranges.param)
pfts = list(param_ranges.pft)
organs = list(param_ranges.organ)

print(param_ranges)

                               param  value_min  value_max  pft  organ
0   fates_maintresp_nonleaf_baserate   0.000002   0.000003    0    NaN
1       fates_mort_understorey_death   0.000000   1.000000    0    NaN
2       fates_phen_drought_threshold  -2.500000  -1.200000    0    NaN
3                   fates_mort_bmort   0.010000   0.060000    1    NaN
4                       fates_grperc   0.110000   0.610000    1    NaN
5                       fates_grperc   0.110000   0.610000    2    NaN
6                 fates_wood_density   0.300000   0.500000    1    NaN
7              fates_leaf_vcmax25top  30.000000  80.000000    1    NaN
8              fates_leaf_vcmax25top  30.000000  80.000000    3    NaN
9        fates_alloc_storage_cushion   1.400000   2.400000    3    NaN
10                 fates_leaf_slatop   0.010000   0.040000    3    NaN
11                fates_wood_density   0.200000   0.900000    3    NaN


In [4]:
n_inst = 120

sampler = qmc.LatinHypercube(d=n_params)
sample = sampler.random(n=n_inst)

# scale to parameter ranges
l_bounds = param_ranges['value_min']
u_bounds = param_ranges['value_max']

scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

print(scaled_sample[0,:])

[ 2.83999314e-06  1.00621538e-01 -2.19663466e+00  4.69569138e-02
  5.58045138e-01  5.59912500e-01  4.68706896e-01  7.57234584e+01
  5.12462969e+01  2.35008616e+00  3.07512111e-02  2.99398720e-01]


In [5]:
## Read in defaut FATES file - note that this is the default for FATES but with:
# - updated allometries for tropical PFTs
# - size bins that are consistent with the DBEN protocol. 
# - supplemental seed rain
# - updated vai bins
# - atkin respiration 
# - size dependent mortality 

input_fname = '/global/homes/j/jneedham/DBEN/Parameter_files/fates_params_bci_base.nc'

# for each sample 
for i in range(0,n_inst) :
    
    # final parameter file name
    fout = '/global/homes/j/jneedham/DBEN/Parameter_files/Ensemble_params/BCI/fates_params_bci_ens_{0}.nc'.format(i)
    
    shutil.copyfile(input_fname, fout)                                                                                                                             
   
    # loop through each parameter and apply either to the correct pft or globally
    for j in range(0, n_params) : 
        
        var = param_names[j]
        pft = pfts[j]
        organ = organs[j]
        
        val = scaled_sample[i, j]
        
        mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
        
        if var == 'fates_mort_bmort' and pft == 1 : 
            pft = pft + 1
            val = val * 0.75
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
            
        if var == 'fates_wood_density' and pft == 1 : 
            pft = pft + 1
            val = val * 1.5
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
            
        if var == 'fates_leaf_vcmax25top' and pft == 1 : 
            pft = pft + 1
            val = val * 0.9
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
            
            
        #parameters that vary as a group (i.e. conifers)
        # if var == 'fates_recruit_seed_alloc' and pft == 0 :
        #     for p in [1,2]:
        #         pft = p
        #         mp.main(var = var, pft = p, fin = fout, val = val, 
        #             fout = fout, O = 1, organ = organ)

            
    

In [6]:
print(scaled_sample.shape)

(120, 12)


In [7]:
tmp = scaled_sample[0:50,:]

df  = pd.DataFrame(data=tmp, columns = param_names)

